## Observation Experiments

_course: quantum cryptography for beginners
<br>date: 10 october 2022
<br>author: burton rosenberg_



### The Observable Class

To organize, definte a Python class `Observable`. Called with the two eigenvalues and eigenvectors, the class builds the projection operators and the observable. It also contains methods to evaluate the probability of outcomes, the expected and standard deviations of the operation.

In [1]:
import math, cmath
import numpy as np
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

class Observable:
    
    def __init__(self,e_val_1,e_vec_1,e_val_2,e_vec_2):
        self.e = (e_val_1,e_val_2)
        self.v = (np.array(e_vec_1),
                  np.array(e_vec_2))
        self.p = (self.outer(self.v[0]),self.outer(self.v[1]))
        self.m = self.e[0]*self.p[0]+self.e[1]*self.p[1]

    def outer(self,v):
        s = np.vdot(v,v)
        w = v.conjugate().reshape((2,1))
        return v*w/s  

    def qf(self,phi,m):
        s = np.vdot(phi,phi)
        phi = phi/cmath.sqrt(s)
        phi_p = np.dot(m,phi)
        return np.vdot(phi,phi_p)
    
    def probs(self,phi):
        return(self.qf(phi,self.v[0]),self.qf(phi,self.v[1]))
    

### The Pauli Matrices

The matrices $X, Y$ and $Z$ are called _Pauli Matrices_,

$$
X = \begin{pmatrix}0&1\\1&0\end{pmatrix},
\;\;
Y = \begin{pmatrix}0&-i\\i&0\end{pmatrix},
\;\;
Z = \begin{pmatrix}1&0\\0&-1\end{pmatrix},
\;\;
$$

and have a number of interesting properties. They are square roots of unity,

$$
X^2 = Y^2 = Z^2 = I
$$
they anti-commute amongst themselves,
$$
XY=-YX,\;  YZ = -ZY \; XZ = - ZX
$$
and they multiply circularly,
$$
XY = iZ, \; YZ = iX,\; ZX = iY
$$

The transformations $H$ and $\tilde{H}$ that assist in the measurent in the $x$ and the $y$ basis are simply,

$$
H = (1/2) (X+Z) \;\; \tilde{H} = (1/2)(Y+Z)
$$


In [2]:
import math
import numpy as np

# Exercise A
# answer 
euler_angles = {
    'zero': (0,0),
    'one' : (math.pi,0),
    'plus': (math.pi/2,0),
    'minus' : (math.pi/2,math.pi),
    'left': (math.pi/2,math.pi/2),
    'right': (math.pi/2,-math.pi/2),
}


# Exercise B
# answer
observables = {
    'Z': (1.0,[1.0,0.0],-1.0,[0.0,1.0]),
    'X': (1.0,[1.0,1.0],-1.0,[1.0,-1.0]),
    'Y': (1.0,[1.0,-1.0j],-1.0,[1.0,1.0j]),
}

## Exercise C
# answer
TH = np.array([1,-1j,1j,-1]).reshape((2,2))/math.sqrt(2)

def all_obs(os):
    r = {}
    for o in os:
        arg = os[o]
        r[o] = Observable(arg[0],arg[1],arg[2],arg[3])
    return r
        
pauli_a = all_obs(observables)
for pauli in pauli_a:
    print(pauli,pauli_a[pauli].m)


Z [[ 1.00  0.00]
 [ 0.00 -1.00]]
X [[0.00 1.00]
 [1.00 0.00]]
Y [[0.00+0.00j 0.00-1.00j]
 [0.00+1.00j 0.00+0.00j]]


### Using the Qiskit Aer simulator



See the Qiskit documentation for the [Aer](https://qiskit.org/documentation/tutorials/simulators/1_aer_provider.html) local simulators. These are faster than going to IBM either for a machine or their simulators, and do not need an internet connection, and do not need to load an account.

In [3]:
import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi


In [4]:
# Create circuit
circ = QuantumCircuit(2)
circ.h(0)
circ.cx(0, 1)
circ.measure_all()
print(circ.draw(output='text'))

# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circ, simulator)

# Run and get counts
result = simulator.run(circ).result()
counts = result.get_counts(circ)
print(counts)

        ┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1 
{'11': 523, '00': 501}


## Measuring along the X, Y and Z axis: experiments

We can measure along the $z$ axis directly, and the $x$ axis by first applying $H$, and the $y$ axis by first applying $\tilde{H}$. You are to derive the formula for a zero in each basis and compare it to experimental results from the simulator. We will test at various theta and phi, 

- in 16 increments of $\theta$ from $0$ to $\pi$;
- 16 increments of $\phi$ from $0$ to $2\pi$.


The __exercise__ is to,

1. Create the measurement. That is, to define a gate that will go before the $Z$ basis measurement, that will transform the $Z$ basis measurement into a $X$ or $Y$ basis measurement.
1. Derive the probabilty of the measurment in terms of $\theta$ and $\phi$. Recall that these parameters define the state by the vector $(\cos \theta/2, e^{i\phi}\sin \theta/2)$.

Here is some common code.

In [9]:
import math, cmath
import numpy as np
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)


def test_meas(make_circuit, calc_probability):
    simulator = Aer.get_backend('aer_simulator')
    #print(simulator.configuration().basis_gates)
    lambda_ = 0.0
    overall_d = 0.0
    for i in range(17):
        phi = i*math.pi/8.0
        print(f'\nphi: {i} pi/8\ntheta\tcalculated\texperimental')
        d = 0.0
        for j in range(17):
            theta = j*math.pi/16.0
            qc = make_circuit(simulator,theta,phi)
            result = simulator.run(qc,shots=1024).result()
            counts = result.get_counts(qc)
            pr_calc = calc_probability(theta,phi)
            if '0' not in counts:
                counts['0'] = 0
            if '1' not in counts:
                counts['1'] = 0 
            pr_expr = counts['0']/(counts['0']+counts['1'])
            print(f'{j} pi/16:\t{pr_calc:.4f}\t{pr_expr:.4f}')
            if abs(pr_expr-pr_calc)>d:
                d = abs(pr_expr-pr_calc)
        print(f'** max deviation: {d:.4f}')
        if d>overall_d:
            overall_d = d
    print(f'** overal max deviation: {overall_d:.4f}')

 

### Z measurement

Since the quantum computer naturally measures in the $Z$ basis, there is nothing to do but measure. 

The sweep of $\theta$ from $0$ to $\pi$, from 1 to 0 with 1/2 at $\theta/2$, independent of $\phi$.

In [12]:
       
def z_probability(theta,phi):
    
    # this is the formula that we have been using.
    # it is for the z basis
    p = math.cos(theta/2.0)**2
    
    return p

def z_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    # prepare test state
    qc.u(theta,phi,0,0)
    
    # measuring on the Z basis
    # need to put the correct gate here
        # qc.?()
        # no circuit needs to go here, 
        # qiskit measures already on the Z basis
    # 
    
    qc.measure_all()
    #print(qc.draw(output='text'))
    return transpile(qc,sim)


test_meas(z_circuit,z_probability)


phi: 0 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	0.9904	0.9912
2 pi/16:	0.9619	0.9609
3 pi/16:	0.9157	0.9043
4 pi/16:	0.8536	0.8564
5 pi/16:	0.7778	0.7734
6 pi/16:	0.6913	0.6885
7 pi/16:	0.5975	0.5977
8 pi/16:	0.5000	0.5107
9 pi/16:	0.4025	0.3770
10 pi/16:	0.3087	0.2998
11 pi/16:	0.2222	0.2227
12 pi/16:	0.1464	0.1650
13 pi/16:	0.0843	0.0771
14 pi/16:	0.0381	0.0400
15 pi/16:	0.0096	0.0117
16 pi/16:	0.0000	0.0000
** max deviation: 0.0255

phi: 1 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	0.9904	0.9912
2 pi/16:	0.9619	0.9678
3 pi/16:	0.9157	0.9141
4 pi/16:	0.8536	0.8594
5 pi/16:	0.7778	0.7734
6 pi/16:	0.6913	0.6934
7 pi/16:	0.5975	0.6260
8 pi/16:	0.5000	0.4863
9 pi/16:	0.4025	0.4238
10 pi/16:	0.3087	0.3213
11 pi/16:	0.2222	0.2334
12 pi/16:	0.1464	0.1582
13 pi/16:	0.0843	0.0645
14 pi/16:	0.0381	0.0352
15 pi/16:	0.0096	0.0117
16 pi/16:	0.0000	0.0000
** max deviation: 0.0284

phi: 2 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 

### X measurement

We consider this measurement by sweeping along $\theta$ (the latitude of the Bloch sphere) for various values of $\phi$ (the longitude of the Bloch sphere).

- When $\phi$ is zero, the sweep of $\theta$ from $0$ to $\pi$, takes us from $|0\rangle$ to $|1\rangle$ through $|+\rangle$. The probability should sweep from 1/2 to 1 back to 1/2.

- When $\phi=\pi$, the sweep of $\theta$ takes us from $|0\rangle$ to $|1\rangle$ through $|-\rangle$.
The probability should sweep from 1/2 to 0 back to 1/2.

- When $\phi=\pm \pi/2$, the sweep of $\theta$ takes us from $|0\rangle$ to $|1\rangle$ through $|\!\circlearrowleft\rangle$ or $|\!\circlearrowright\rangle$ .
The probability should be a constant 1/2.


Hint: The [H gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html#qiskit.circuit.library.HGate)


In [11]:

def x_probability(theta,phi):

    p = 1.0 # need to put the correct formula here
    
    return p
    

def x_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    # prepare test state
    qc.u(theta,phi,0,0)
    
    # measuring on the X basis
    # need to put the correct gate here
        # qc.?()
    #
    
    qc.measure_all()

    return transpile(qc,sim)

test_meas(x_circuit,x_probability)


phi: 0 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	1.0000	0.9922
2 pi/16:	1.0000	0.9707
3 pi/16:	1.0000	0.9053
4 pi/16:	1.0000	0.8477
5 pi/16:	1.0000	0.7754
6 pi/16:	1.0000	0.7158
7 pi/16:	1.0000	0.6240
8 pi/16:	1.0000	0.4814
9 pi/16:	1.0000	0.3936
10 pi/16:	1.0000	0.3105
11 pi/16:	1.0000	0.2188
12 pi/16:	1.0000	0.1582
13 pi/16:	1.0000	0.0879
14 pi/16:	1.0000	0.0420
15 pi/16:	1.0000	0.0078
16 pi/16:	1.0000	0.0000
** max deviation: 1.0000

phi: 1 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 pi/16:	1.0000	0.9893
2 pi/16:	1.0000	0.9707
3 pi/16:	1.0000	0.9189
4 pi/16:	1.0000	0.8594
5 pi/16:	1.0000	0.8037
6 pi/16:	1.0000	0.6826
7 pi/16:	1.0000	0.5732
8 pi/16:	1.0000	0.5078
9 pi/16:	1.0000	0.3975
10 pi/16:	1.0000	0.3174
11 pi/16:	1.0000	0.2031
12 pi/16:	1.0000	0.1533
13 pi/16:	1.0000	0.0918
14 pi/16:	1.0000	0.0439
15 pi/16:	1.0000	0.0088
16 pi/16:	1.0000	0.0000
** max deviation: 1.0000

phi: 2 pi/8
theta	calculated	experimental
0 pi/16:	1.0000	1.0000
1 

### Y measurement

We derived that using a $\tilde{H}$ in front of a $Z$ measure gives a $Y$ measurement. This will measure similar to the $X$ measurement, except that the roles of left and right replace those of plus and minus.

- When $\phi=\pi/2$, the sweep of $\theta$ from $0$ to $\pi$, takes us from $|0\rangle$ to $|1\rangle$ through $|\!\circlearrowleft\rangle$. The probability should sweep from 1/2 to 1 back to 1/2.

- When $\phi=3\pi/2$, the sweep of $\theta$ takes us from $|0\rangle$ to $|1\rangle$ through $|\!\circlearrowright\rangle$. The probability should sweep from 1/2 to 0 back to 1/2.

- When $\phi=0$ or $\phi=\pi$, the sweep of $\theta$ takes us from $|0\rangle$ to $|1\rangle$ through $|+\rangle$ or $|-\rangle$. The probability should be a constant 1/2.

Hint: The [U2 gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.U2Gate.html)

In [8]:


def y_probability(theta,phi):
    
    p = 1.0 # need to put the correct formula here
    
    return p


def y_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    qc.u(theta,phi,0,0)
    
    # measuring on the Y basis
    # need to put the correct gate here
        # qc.?()
    # 
   
    qc.measure_all()
    return transpile(qc,sim)

test_meas(y_circuit,y_probability)


phi: 0 pi/8
theta
0 pi/16:	0.5000	0.5098
1 pi/16:	0.5000	0.5059
2 pi/16:	0.5000	0.4971
3 pi/16:	0.5000	0.4922
4 pi/16:	0.5000	0.4678
5 pi/16:	0.5000	0.4629
6 pi/16:	0.5000	0.5000
7 pi/16:	0.5000	0.4883
8 pi/16:	0.5000	0.4766
9 pi/16:	0.5000	0.5117
10 pi/16:	0.5000	0.5010
11 pi/16:	0.5000	0.5098
12 pi/16:	0.5000	0.5098
13 pi/16:	0.5000	0.4707
14 pi/16:	0.5000	0.4844
15 pi/16:	0.5000	0.5225
16 pi/16:	0.5000	0.5127
** max deviation: 0.0371

phi: 1 pi/8
theta
0 pi/16:	0.5000	0.5078
1 pi/16:	0.5373	0.5498
2 pi/16:	0.5732	0.5850
3 pi/16:	0.6063	0.6328
4 pi/16:	0.6353	0.6328
5 pi/16:	0.6591	0.6719
6 pi/16:	0.6768	0.6826
7 pi/16:	0.6877	0.7012
8 pi/16:	0.6913	0.6875
9 pi/16:	0.6877	0.6914
10 pi/16:	0.6768	0.6826
11 pi/16:	0.6591	0.6553
12 pi/16:	0.6353	0.6357
13 pi/16:	0.6063	0.6025
14 pi/16:	0.5732	0.5713
15 pi/16:	0.5373	0.5762
16 pi/16:	0.5000	0.5107
** max deviation: 0.0388

phi: 2 pi/8
theta
0 pi/16:	0.5000	0.4980
1 pi/16:	0.5690	0.5742
2 pi/16:	0.6353	0.6299
3 pi/16:	0.6964	0.7109
4 pi/

/var/folders/v7/bhvmfjbd1svd5qtsfjxhbk3m0000gn/T/ipykernel_33094/2133213139.py:7: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  qc.u2(math.pi/2,math.pi/2,0)


16 pi/16:	0.5000	0.4854
** max deviation: 0.0254

phi: 3 pi/8
theta
0 pi/16:	0.5000	0.4971
1 pi/16:	0.5901	0.5752
2 pi/16:	0.6768	0.6807
3 pi/16:	0.7566	0.7402
4 pi/16:	0.8266	0.8057
5 pi/16:	0.8841	0.8779
6 pi/16:	0.9268	0.9268
7 pi/16:	0.9531	0.9619
8 pi/16:	0.9619	0.9697
9 pi/16:	0.9531	0.9541
10 pi/16:	0.9268	0.9326
11 pi/16:	0.8841	0.8711
12 pi/16:	0.8266	0.8340
13 pi/16:	0.7566	0.7344
14 pi/16:	0.6768	0.6670
15 pi/16:	0.5901	0.5830
16 pi/16:	0.5000	0.4932
** max deviation: 0.0223

phi: 4 pi/8
theta
0 pi/16:	0.5000	0.4961
1 pi/16:	0.5975	0.5820
2 pi/16:	0.6913	0.7100
3 pi/16:	0.7778	0.7539
4 pi/16:	0.8536	0.8379
5 pi/16:	0.9157	0.9268
6 pi/16:	0.9619	0.9629
7 pi/16:	0.9904	0.9902
8 pi/16:	1.0000	1.0000
9 pi/16:	0.9904	0.9893
10 pi/16:	0.9619	0.9629
11 pi/16:	0.9157	0.9150
12 pi/16:	0.8536	0.8779
13 pi/16:	0.7778	0.7783
14 pi/16:	0.6913	0.6699
15 pi/16:	0.5975	0.5771
16 pi/16:	0.5000	0.4932
** max deviation: 0.0244

phi: 5 pi/8
theta
0 pi/16:	0.5000	0.5098
1 pi/16:	0.5901	0.6064
2 

### Answers


In [ ]:
def z_probability(theta,phi):
    return math.cos(theta/2.0)**2

def z_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    qc.u(theta,phi,0,0)

    qc.measure_all()
    return transpile(qc,sim)

def x_probability(theta,phi):
    return (0.5)*(1.0+math.sin(theta)*math.cos(phi))

def x_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    qc.u(theta,phi,0,0)
    
    qc.h(0)  # rotate + and 0, - and 1
    
    qc.measure_all()
    return transpile(qc,sim)

def y_probability(theta,phi):
    return (0.5)*(1.0+math.sin(theta)*math.sin(phi))

def y_circuit(sim,theta,phi):
    qc = QuantumCircuit(1)
    qc.u(theta,phi,0,0)
    
    qc.u2(math.pi/2,math.pi/2,0) # rotate left and 0, right and 1
    
    qc.measure_all()
    return transpile(qc,sim)


#### END